# 1.Import Data

In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
data = np.abs(round(pd.read_csv('Data/data2.csv'),6))
data

,prev_displ,prev_vell,prev_accl,prev_wspd,displ,vell,accl,wspd,act,dispm,velm,accm,dispm_0,velm_0,accm_0
0,0.000000,0.000000,0.000000,0.000,0.000101,0.000005,0.000060,2.860,10.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336
1,0.000000,0.000000,0.000000,0.000,0.000100,0.000018,0.000051,2.860,20.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336
2,0.000000,0.000000,0.000000,0.000,0.000099,0.000025,0.000035,2.860,30.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336
3,0.000000,0.000000,0.000000,0.000,0.000098,0.000026,0.000017,2.860,40.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336
4,0.000000,0.000000,0.000000,0.000,0.000096,0.000022,0.000006,2.860,50.0,0.000107,0.000146,0.000336,0.000108,0.000142,0.000336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18422,0.000099,0.000004,0.000133,4.444,0.000099,0.000004,0.000133,3.168,30.0,0.000135,0.000296,0.001199,0.000156,0.000392,0.001209
18423,0.000103,0.000013,0.000172,4.444,0.000103,0.000013,0.000172,3.168,40.0,0.000133,0.000273,0.001199,0.000156,0.000392,0.001209
18424,0.000109,0.000018,0.000227,4.444,0.000109,0.000018,0.000227,3.168,50.0,0.000134,0.000258,0.001199,0.000156,0.000392,0.001209
18425,0.000115,0.000016,0.000315,4.444,0.000115,0.000016,0.000315,3.168,60.0,0.000136,0.000251,0.001199,0.000156,0.000392,0.001209


In [ ]:
#reward function = Ucondi/Uuncondi if 
for j in ['disp','vel','acc']:
    data['r_'+j] = np.abs(data[j+'m']/data[j+'m_0'])
    # if no change reward = 0
    data.loc[data['r_'+j]==1,'r_'+j] = 0 - data['act']/2000
    data.loc[data['r_'+j]>1,'r_'+j] = -data.loc[data['r_'+j]>1,'r_'+j] - data['act']/2000
    data.loc[(data['r_'+j]<1)&(data['r_'+j]>0),'r_'+j] = 1/data.loc[data['r_'+j]<1,'r_'+j] - data['act']/2000
    
    

In [ ]:
data.head()

,prev_displ,prev_vell,prev_accl,prev_wspd,displ,vell,accl,wspd,act,dispm,velm,accm,dispm_0,velm_0,accm_0,r_disp,r_vel,r_acc
0,0.0,0.0,0.0,0.0,0.000101,0.000005,0.000060,2.86,10.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336,1.004346,-0.005000,-0.005
1,0.0,0.0,0.0,0.0,0.000100,0.000018,0.000051,2.86,20.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336,0.999346,-0.010000,-0.010
2,0.0,0.0,0.0,0.0,0.000099,0.000025,0.000035,2.86,30.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336,0.994346,-0.015000,-0.015
3,0.0,0.0,0.0,0.0,0.000098,0.000026,0.000017,2.86,40.0,0.000107,0.000142,0.000336,0.000108,0.000142,0.000336,0.989346,-0.020000,-0.020
4,0.0,0.0,0.0,0.0,0.000096,0.000022,0.000006,2.86,50.0,0.000107,0.000146,0.000336,0.000108,0.000142,0.000336,0.984346,-1.053169,-0.025


# 2. Create Subset data

## 2.1 Check Data summary for rewards

In [ ]:
for j in ['disp','vel','acc']:
    print(data['r_'+j].mean())
    print(data['r_'+j].median())
    print()

0.8163813642101445
1.1428042328042327

1.2900314900556042
1.466413612565445

0.2540489646188444
0.9467380720545279



# 2.2 Separate the Data for models

In [ ]:
for j in ['disp','vel','acc']:
    col_names = ['prev_'+j+'l', 'prev_wspd', j+'l', 'wspd', 'act', 'r_'+j]
    globals()[j] = round(data[col_names],6)
    eval(j)['state'] = list(zip( np.abs(eval(j)['prev_'+j+'l']),  eval(j)['prev_wspd']))
    eval(j)['state_p'] = list(zip( np.abs(eval(j)[j+'l']),  eval(j)['wspd']))
    globals()[j] = eval(j)[['state', 'state_p', 'act', 'r_'+j]].drop_duplicates()
    eval(j).to_csv('Data/{}.csv'.format(j))
# we have three datasets: disp, vel, acc, with s, s', a r
disp

,state,state_p,act,r_disp
0,"(0.0, 0.0)","(0.000101, 2.86)",10.0,1.004346
1,"(0.0, 0.0)","(0.0001, 2.86)",20.0,0.999346
2,"(0.0, 0.0)","(9.9e-05, 2.86)",30.0,0.994346
3,"(0.0, 0.0)","(9.8e-05, 2.86)",40.0,0.989346
4,"(0.0, 0.0)","(9.6e-05, 2.86)",50.0,0.984346
...,...,...,...,...
18422,"(9.9e-05, 4.444)","(9.9e-05, 3.168)",30.0,1.140556
18423,"(0.000103, 4.444)","(0.000103, 3.168)",40.0,1.152932
18424,"(0.000109, 4.444)","(0.000109, 3.168)",50.0,1.139179
18425,"(0.000115, 4.444)","(0.000115, 3.168)",60.0,1.117059


In [ ]:
#check the total states vs observed next states
for j in [disp,vel,acc]:
    print('total samples:',len(j))
    print('total unique current state:', len(set(j['state'])))
    print('total unique next state:',len(set(j['state_p'])))
    print()

total samples: 16984
total unique current state: 4601
total unique next state: 2351

total samples: 17721
total unique current state: 4250
total unique next state: 2212

total samples: 16152
total unique current state: 4522
total unique next state: 2417



In [ ]:
for i in data.columns:
    print(i,':',len(set(data[i])))
# 20 actions
# 395 displacement
# 675 velocity
# 1026 acceleration

prev_displ : 206
prev_vell : 398
prev_accl : 788
prev_wspd : 139
displ : 206
vell : 398
accl : 788
wspd : 139
act : 20
dispm : 217
velm : 686
accm : 2133
dispm_0 : 220
velm_0 : 523
accm_0 : 754
r_disp : 14805
r_vel : 17183
r_acc : 14443


## 2.3 Unique Value of (parameter, wind_speed)

In [ ]:
for i in ['prev_displ','prev_vell','prev_accl']:
    temp = set(data[i])
    act = set(data['wspd'])
    globals()['unique_{}'.format(i)] = [(j,k) for j in temp for k in act]
len(unique_prev_displ)

28634

# 4. Dictionarized all (disp/vel/acc, wind) to s1,s2...

In [ ]:
#create dictionary for each state
for i in ['unique_prev_displ','unique_prev_vell','unique_prev_accl']:
  my_list = eval(i)
  globals()['dict_'+i] = dict() 
  for index,value in enumerate(my_list):
    eval('dict_'+i)[index] = value
#dict_unique_prev_displ
len(dict_unique_prev_displ.values())

28634

## 4.1 Conver the tuple states into a dictionarized states 

In [ ]:
for i in ['disp','vel','acc']:
    temp = eval(i)
    dic_i = eval('dict_unique_prev_'+i+'l')
    key_list = list(dic_i.keys())
    val_list = list(dic_i.values())

    temp['s'] = 0
    temp['s_p'] = 0

    for i in range(len(temp)):
        j = temp['state'].iloc[i]
        k = temp['state_p'].iloc[i]
        position_s = val_list.index(j)
        position_s_p = val_list.index(k)
        temp['s'].iloc[index] = key_list[position_s]
        temp['s_p'].iloc[index] = key_list[position_s_p]
        
    globals()[i] = temp
#my_dict_unique_displ.get()

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
disp

,state,state_p,act,r_disp,s,s_p
0,"(0.0, 0.0)","(0.000101, 2.86)",10.0,1.004346,25,11957
1,"(0.0, 0.0)","(0.0001, 2.86)",20.0,0.999346,25,1532
2,"(0.0, 0.0)","(9.9e-05, 2.86)",30.0,0.994346,25,19046
3,"(0.0, 0.0)","(9.8e-05, 2.86)",40.0,0.989346,25,8899
4,"(0.0, 0.0)","(9.6e-05, 2.86)",50.0,0.984346,25,15988
...,...,...,...,...,...,...
18422,"(9.9e-05, 4.444)","(9.9e-05, 3.168)",30.0,1.140556,19145,19076
18423,"(0.000103, 4.444)","(0.000103, 3.168)",40.0,1.152932,4828,4759
18424,"(0.000109, 4.444)","(0.000109, 3.168)",50.0,1.139179,11083,11014
18425,"(0.000115, 4.444)","(0.000115, 3.168)",60.0,1.117059,17199,17130


In [ ]:
vel

,state,state_p,act,r_vel,s,s_p
0,"(0.0, 0.0)","(5e-06, 2.86)",10.0,-0.005000,25,11818
1,"(0.0, 0.0)","(1.8e-05, 2.86)",20.0,-0.010000,25,9038
2,"(0.0, 0.0)","(2.5e-05, 2.86)",30.0,-0.015000,25,3339
3,"(0.0, 0.0)","(2.6e-05, 2.86)",40.0,-0.020000,25,50321
4,"(0.0, 0.0)","(2.2e-05, 2.86)",50.0,-1.053169,25,29332
...,...,...,...,...,...,...
18422,"(4e-06, 4.444)","(4e-06, 3.168)",30.0,1.309324,20674,20605
18423,"(1.3e-05, 4.444)","(1.3e-05, 3.168)",40.0,1.415897,52922,52853
18424,"(1.8e-05, 4.444)","(1.8e-05, 3.168)",50.0,1.494380,9137,9068
18425,"(1.6e-05, 4.444)","(1.6e-05, 3.168)",60.0,1.531753,26512,26443


In [ ]:
acc

,state,state_p,act,r_acc,s,s_p
0,"(0.0, 0.0)","(6e-05, 2.86)",10.0,-0.00500,25,61997
1,"(0.0, 0.0)","(5.1e-05, 2.86)",20.0,-0.01000,25,106616
2,"(0.0, 0.0)","(3.5e-05, 2.86)",30.0,-0.01500,25,54213
3,"(0.0, 0.0)","(1.7e-05, 2.86)",40.0,-0.02000,25,35865
4,"(0.0, 0.0)","(6e-06, 2.86)",50.0,-0.02500,25,6258
...,...,...,...,...,...,...
18422,"(0.000133, 4.444)","(0.000133, 3.168)",30.0,0.99334,8442,8373
18423,"(0.000172, 4.444)","(0.000172, 3.168)",40.0,0.98834,102545,102476
18424,"(0.000227, 4.444)","(0.000227, 3.168)",50.0,0.98334,30821,30752
18425,"(0.000315, 4.444)","(0.000315, 3.168)",60.0,0.97834,46528,46459


# 4.2 Save the Data to Data file
* csv
* Dictionary

In [ ]:
for i in ['disp','vel','acc']:
    eval(i).to_csv('Data/{}.csv'.format(i))

    dic_i = eval('dict_unique_prev_'+i+'l')
    a_file = open("Data/{}_dic.json".format(i), "w")
    json.dump(dic_i, a_file)
    a_file.close()

### Loading Test

In [ ]:
for i in ['disp','vel','acc']:
    a_file = open("Data/{}_dic.json".format(i), "r")
    output = a_file.read()
    globals()['{}_dic'.format(i)]=output

In [ ]:
# ratio =  max(DO) / max(NOTHING)
    # if ratio = 1 -> reward = 0 + penalty
    # if ratio > 1 -> reward = -ratio + penalty
    # if ratio < 1 -> reward = 1/ratio + penalty

# reward = max(NOTHING) / max(DO) - Voltage / constant 